In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics

In [2]:
df = pd.read_csv("dataset.csv")

In [3]:
df.head()

Unnamed: 0  Collision_Ref_No  Policing_Area  Speed_Limit  Junction_Detail  \
0           0              3518             21           60              1.0   
1           1             10557             13           50             12.0   
2           2              5002              2           60             12.0   
3           4             12416              7           60              6.0   
4           5               558             15           60              1.0   

   Ped_Crossing_PC  Light_Conditions  Weather_Conditions  \
0              1.0                 2                   9   
1              1.0                 4                   3   
2              1.0                 2                   2   
3              1.0                 2                   3   
4              1.0                 2                   2   

   Road_Surface_Conditions  Collision_Severity  
0                      9.0                   3  
1                      1.0                   3  
2                      2.0                   3  
3                      1.0                   3  
4                      2.0                   2

In [4]:
colnames_numeric = df.columns[0:-1]

In [5]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df[colnames_numeric] = scaler.fit_transform(df[colnames_numeric])

In [6]:
df.head()

Unnamed: 0  Collision_Ref_No  Policing_Area  Speed_Limit  Junction_Detail  \
0    0.000000          0.267255       0.512821     0.833333         0.000000   
1    0.000113          0.802907       0.307692     0.666667         1.000000   
2    0.000226          0.380184       0.025641     0.833333         1.000000   
3    0.000452          0.944373       0.153846     0.833333         0.454545   
4    0.000565          0.042006       0.358974     0.833333         0.000000   

   Ped_Crossing_PC  Light_Conditions  Weather_Conditions  \
0              0.0          0.166667            0.888889   
1              0.0          0.500000            0.222222   
2              0.0          0.166667            0.111111   
3              0.0          0.166667            0.222222   
4              0.0          0.166667            0.111111   

   Road_Surface_Conditions  Collision_Severity  
0                 0.888889                   3  
1                 0.000000                   3  
2                 0.111111                   3  
3                 0.000000                   3  
4                 0.111111                   2

In [7]:
df.shape

(6940, 10)

In [8]:
df = df.values.tolist()

In [9]:
import random
def train_test_split(data, split, trainingSet = [], testSet = []):
    for x in range(len(data)):
        if random.random() < split:
            trainingSet.append(data[x])
        else:
            testSet.append(data[x])

In [10]:
trainingSet = []
testSet = []
split = 0.70
train_test_split(df, split, trainingSet, testSet)

In [11]:
len(trainingSet)

4775

In [12]:
len(testSet)

2165

In [13]:
import math
def Euclideandist(x,xi, length):
    d = 0.0
    for i in range(length):
        d += pow(float(x[i])- float(xi[i]),2)
    return math.sqrt(d)

In [14]:
#Getting the K neighbours having the closest Euclidean distance to the test instance
import operator
def getNeighbors(trainingSet, testInstance, k):
    distances = []
    length = len(testInstance)-1
    for x in range(len(trainingSet)):
        dist = Euclideandist(testInstance, trainingSet[x], length)
        distances.append((trainingSet[x], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors

In [15]:
import operator
def getResponse(neighbors):
    classVotes = {}
    for x in range(len(neighbors)):
        response = neighbors[x][-1]
        if response in classVotes:
            classVotes[response] += 1
        else:
            classVotes[response] = 1
    sortedVotes = sorted(classVotes.items(), key=operator.itemgetter(1), reverse=True)
    return sortedVotes[0][0]

In [16]:
def getAccuracy(testSet, predictions):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][-1] == predictions[x]:
            correct += 1
    return (correct/float(len(testSet))) * 100.0

In [17]:
def test_df(testSet):
    y_test=[]
    for i in range(len(testSet)):
        t=testSet[i][-1]
        y_test.append(t)
    return y_test

In [18]:
y_test=test_df(testSet)

In [19]:
predictions=[]
k = 3
for x in range(len(testSet)):
    neighbors = getNeighbors(trainingSet, testSet[x], k)
    result = getResponse(neighbors)
    predictions.append(result)

In [20]:
accuracy = getAccuracy(testSet, predictions)
Acc=repr(accuracy)
print("KNN",Acc)

KNN 86.83602771362587


In [21]:
metrics.precision_score(y_test, predictions,average="weighted")

0.8259540944189513

In [22]:
print('Precision:', metrics.precision_score(y_test, predictions,average="weighted")*100)
print('Recall:', metrics.recall_score(y_test, predictions,average="weighted")*100)
print('F1-Score:', metrics.f1_score(y_test, predictions,average="weighted")*100)

Precision: 82.59540944189513
Recall: 86.83602771362587
F1-Score: 84.34083020815234


In [23]:
ACCURACY = metrics.accuracy_score(y_test, predictions)*100
PRECISION = metrics.precision_score(y_test, predictions,average="weighted")*100
RECALL = metrics.recall_score(y_test, predictions,average="weighted")*100
F1_SCORE = metrics.recall_score(y_test, predictions,average="weighted")*100

In [24]:
data=pd.DataFrame([['KNN',ACCURACY,PRECISION,RECALL,F1_SCORE]],columns=['Model','ACCURACY','PRECISION','RECALL','F1-SCORE'])
data.to_csv('scores_knn.csv',index=False)

In [25]:
data

Model   ACCURACY  PRECISION     RECALL   F1-SCORE
0   KNN  86.836028  82.595409  86.836028  86.836028